In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from glob import glob

import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import layers

In [ ]:
# os.listdir('train')

In [ ]:
from zipfile import ZipFile
# with ZipFile('../input//aerial-cactus-identification.zip')as zip_obj :
#   zip_obj.extractall()
with ZipFile('../input/aerial-cactus-identification/test.zip')as test_obj :
  test_obj.extractall()
with ZipFile('../input/aerial-cactus-identification/train.zip')as train_obj :
  train_obj.extractall()

## 데이터 확인

## csv파일 확인
 - ```train.csv``` : id, has_cactus가 담겨 있으며 총 17500개의 데이터
  - train데이터에 관한 csv
 - ```sample_submission.csv``` : 구성은 같으며 4000개의 데이터
  - test데이터에 관한 csv

In [ ]:
df = pd.read_csv('../input/aerial-cactus-identification/train.csv')
print(df.head())

file_list = df['id']
has_cactus = df['has_cactus']
print(len(file_list), len(has_cactus))

In [ ]:
test_df = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
print(test_df.head())

test_fnames = test_df['id']
test_labels = test_df['has_cactus']

print(len(test_fnames), len(test_labels))

In [ ]:
data_paths = glob('train/*.jpg')
test_paths = glob('test/*.jpg')
print(len(data_paths), len(test_paths))

In [ ]:
pa = glob('train/*.jpg')[0]
pa
g = tf.io.read_file(pa)
im = tf.io.decode_image(g)
print(im.shape)
plt.imshow(im)
plt.show()

In [ ]:
input_shape = (32, 32, 3)
batch_size = 32

하지만 케글에서 실습을 하면서 강사님께서 알려주신 방법은 다음과 같다  
csv에서 ```id``` 열의 데이터를 활용해서 파일의 경로를 만들고 그에따른 label값을 ```has_cactus```열의 데이터로 만들었음

In [ ]:
data_dir = 'train'

data_paths = []
for fname, label in zip(file_list, has_cactus) :
  data_paths.append((os.path.join(data_dir, fname), label))

In [ ]:
data_paths[:10]

### ***학습을 할때 validatio_data가 있으면 좋은 것일까?***
그전에 먼저 데이터에 대해서 알아본다  
<br>
#### 데이터의 종류
- Train Data
 - 분석 모델을 만들기 위한 학습용 데이터이다
- Validation Data
 - 여러 분석 모델 중 어떤 모델이 적합한지 선택하기 위한 검증용 데이터이다.
- Test Data
 - 최종적으로 선택된 분석 모델이 얼마나 잘 작동하는지 확인하기 위한 결과용 데이터이다.<br>

#### https://3months.tistory.com/118




```python
def read_data(path) :
  gfile = tf.io.read_file(path)
  image = tf.io.decode_image(gfile)

  class_name = tf.strings.split(path, '/')[-1]
  class_name = tf.strings.split(class_name, '_')[-1]
  class_name = tf.strings.split(class_name, '.')[0]

  onehot = tf.cast(labels == class_name, tf.uint8)

  return image, onehot
```

기존의 read_data는 위의 함수의 형태로 되어 있었다. (mnist, cifar10)  
mnist와 cifar와 같은 경우는 10개의 클래스로 분류하는 것
하지만 이 데이터는
- 입력된 사진으로 부터
  - 선인장이 있는지
  - 없는지
2가지로 분류하는 것  

- 이진분류? 
 - 이진 분류에서 많이 사용하는 활성화함수 : ```sigmoid```
 - 다중 분류에서 많이 사용되는 활성화함수 : ```softmax```
 - 추가적으로 이진분류의 loss는 ```binary_crossentroy``` 활용

In [ ]:
def tmp_func (path_name) :
  return path_name

In [ ]:
def read_data(path_name) :
  img_path = path_name[0]
  label = tf.strings.to_number(path_name[1], out_type=tf.int64)

  gfile = tf.io.read_file(img_path)
  image = tf.io.decode_image(gfile)
  
  return image, label

In [ ]:
a = tf.data.Dataset.from_tensor_slices(np.array(data_paths[:2]))
a = a.map(tmp_func)
p = next(iter(a))
p[0], p[1]

이전에 만든 data_path에는 경로와 그에 따른 label값들이 들어가게 된다  
이를 각각 경로를 통해서 image를 읽어들이는 것과 label값을 return하도록 만든다  
<br>
**근데 .... label값을 그대로 return 해줘도 되는가?**  
<br>
이게 제일 의문점이다.  
image 자체는 ```tf.io.read_file```, ```tf.io.decode_image``` 기존에 사용했던 것들을 활용하면 되는 것  
이전에 mnist나 cifar10 같은 경우는 각 입력값들로부터 onehot encoding을 통해서 결과를 return해주었지만 여기선 이진 분류이기 때문에 **선인장이 있는가? 없는가?**를 판단 하는 것이고 ......

[shuffle에서 buffer_size](https://helloyjam.github.io/tensorflow/buffer-size-in-shuffle/)

이미지를 읽고 프로세싱하고 배치작업등의 무거운 작업을 하기 전에 tf.data.Dataset.shuffle()을 호출해야 한다  
+. ```tf.data.Dataset.from_tensor_slices((파일목록, 라벨값))```
 - 파일목록과 라벨값으로 데이터셋을 만드는 것.... 기억해두자

In [ ]:
train_ratio = 0.8

train_paths = data_paths[:int(train_ratio*len(data_paths))]
test_paths = data_paths[int(train_ratio*len(data_paths)):]

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(np.array(train_paths))
train_ds = train_ds.map(read_data)
# print(next(iter(train_ds)))
# 이때 확인하면 image에 대한 tensor값과 그 이미지에 대한 라벨을 알 수 있음
train_ds = train_ds.shuffle(len(train_paths))
# shuffle에 대한 buffer_size에 관해서도(위에 참고)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.repeat()

In [ ]:
valid_ds = tf.data.Dataset.from_tensor_slices(np.array(test_paths))
valid_ds = valid_ds.map(read_data)
valid_ds = valid_ds.batch(batch_size)
valid_ds = valid_ds.repeat()

In [ ]:
inputs = layers.Input(input_shape)

# Feature extraction
net = layers.Conv2D(32, 3, 1, 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, 3, 1, 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2, 2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Conv2D(64, 3, 1, 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, 3, 1, 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2, 2))(net)
net = layers.Dropout(0.5)(net)

# classification
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(1)(net)
net = layers.Activation('sigmoid')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='cactus_cnn')

In [ ]:
model.summary()

In [ ]:
model.compile(loss = tf.keras.losses.binary_crossentropy,
              optimizer = tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

In [ ]:
hist = model.fit(train_ds,
                 validation_data=valid_ds,
                 validation_steps=validation_steps,
                 steps_per_epoch=steps_per_epoch,
                 epochs = 30)

# 평가

In [ ]:
test_df.head()

In [ ]:
test_fnames[:5]

In [ ]:
test_labels[:5]

In [ ]:
test_dir = 'test'

eval_paths = []
for fname in test_fnames :
  eval_paths.append(os.path.join(test_dir, fname))

print(eval_paths[:5])

In [ ]:
def image_read(path) :
  g = tf.io.read_file(path)
  im = tf.io.decode_image(g)

  return im

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
# test_images = [image_read(path) for path in eval_paths]

test_images = []
for path in tqdm_notebook(eval_paths) :
  test_images.append(image_read(path))

# np.array(test_image).shape

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices(eval_paths)
test_ds = test_ds.map(image_read)
test_ds = test_ds.batch(batch_size)

In [ ]:
pred = model.predict(test_ds)

In [ ]:
pred.shape

In [ ]:
pred = pred.reshape((4000))

In [ ]:
submit_df = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
test_fnames = submit_df['id']
test_labels = pred  # 결과가 onehot이 아닌 binary로 담아줘야함

submit_file = pd.DataFrame({'id':test_fnames, 'has_cactus':test_labels}, columns = ['id', 'has_cactus'])
submit_file
submit_file.to_csv('submission.csv', index=False)